In [1]:
# !pip install spacy

from dotenv import load_dotenv

load_dotenv()

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andriimyronenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd

test_tech_df = pd.read_csv('./data/merged/test_df_tech_only.csv')

In [3]:
print(test_tech_df.at[0, 'text'])
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
print(test_tech_df.at[0, 'skills'])
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
print(test_tech_df.at[0, 'knowledge'])

Full Stack Software Engineer - Java / JavaScript <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> . <ADDRESS> <ADDRESS> <LOCATION> - <LOCATION> Date posted: 2021-03-04 Likes: 0 Dislikes: 0 Love: 0 Salary: <SALARY> Job type: FULL_TIME Experience level: <EXPERIENCE> Industry: <INDUSTRY> Company size: <SIZE> Company type: <COMPANY_TYPE> Technologies: javascript reactjs java Job description: Job type: Full-time Role: Full Stack Developer Technologies javascript reactjs java Job description As a member of our <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> we look first and foremost for people who are passionate around solving business problems through innovation and engineering practices . You'll be required to apply your depth of knowledge and expertise to all aspects of the software development lifecycle as well as partner continuously with your many stakeholders on a daily basis to stay focused on common goals . We embrace a culture of experimentation and constantly strive for i

In [4]:
test_tech_df.shape

(31, 5)

In [5]:
verbose_template = """
You are a top-notch recruiter and data labeler.
Your task is to thoroughly analyze the following part of a job posting and extract text spans including skills or knowledge.

You should use the following guideline for annotation:

Instruction for Annotating Text for Skills and Knowledge Components
Objective: Identify and annotate spans of text in job postings (JPs) that represent specific skills or knowledge required for the position.

1. Identifying Skills:

General Rule: A skill is usually indicated by a VERB or an (ADJECTIVE) + NOUN combination.
Exclusion of Modal Verbs: Do not tag modal verbs (e.g., can, will) as part of the skill.
Phrase Separation: Split phrases containing prepositions and/or conjunctions, except when conjunctions coordinate two nouns as a single argument.
Handling Anaphoric Pronouns: Avoid tagging skills with anaphoric pronouns. Only tag the preceding skill.
Splitting Coordinated Components: Split nouns and adjectives in coordination if they lack a verb.
Listing of Skills: If skills are listed leading to different subtasks, annotate each separately.
Brevity in Contextual Information: Keep skill annotations concise, especially when followed by company-specific info.
Inclusion of 'skills' or 'knowledge' in Tags: Include these words in the annotation if their omission alters the meaning.
Parenthetical Information: Include if it elaborates on the skill or is an abbreviation.
Adverbial Inclusion: Include adverbials that describe the manner of doing something.
Attitudes as Skills: Annotate attitudes as skills, omitting articles.
Exclusions:
Avoid tagging ironic skills.
Avoid nested annotations; use one span for overlapping skills.
Do not tag skills in top headlines, but do tag in sub-headlines and body text.
Exclude fluff and triggers surrounding the skill component.
Exclude participation, contributing, and transfer expressions from skill annotations.
Do not annotate occupations or positions.
Focus on skills related and specific to the position.

2. Identifying Knowledge:
General Rule: Knowledge is non-executable and possessed by an individual.
Inclusion of Parenthetical Information: Include if it is related to the knowledge component.
Licenses and Certifications: Include additional words like "certificate," "license," etc.
Vague Preceding Verbs: Only tag the knowledge component if preceded by a vague verb.
Specificity: Annotate only specified knowledge components.
Nested Knowledge in Skills: Knowledge components can be nested within skill annotations.
Coordination of Knowledge Components: Annotate as one if all components share one knowledge tag.
Listing Knowledge Tags: Annotate all knowledge tags separately.
Annotation of Industries and Fields: Tag these as knowledge components.

3. Prioritization and Other Considerations:
Prefer skills over knowledge in uncertain cases.
Prioritize skills over attitudes; only tag the skill within an attitude.
Keep annotations concise and relevant to the job.
Annotate skills and knowledge in unconventional places if related to the position.
Consider annotating a combination of skill and knowledge when applicable.
Exclude knowledge/skill components in positions.
Focus on annotations relevant to the specific position and its future expectations.

Output just the JSON array, wihtout any other text. 
Start your response with "[" symbol, and finish with "]".
Make sure your response doesn't include anything else.

Job posting:
{job_posting}

Response:"""

short_template = """
    You are a top-notch recruiter and data labeler.
    Your task is to thoroughly analyze the following part of a job posting and extract text spans including skills and knowledge.
    A skill in a job context is typically indicated by a verb or an adjective-noun combination, reflecting an executable ability or a specific way of performing a task. 
    Knowledge, on the other hand, refers to non-executable information that an individual possesses, often indicated by specific fields, industries, or certifications, and is distinct from direct action or skills.

    Output the JSON array, wihtout any other text. 
    Start your response with "[" symbol, and finish with "]".
    Make sure your response doesn't include anything else.
        
    Job posting:
    {job_posting}
"""

few_shot_template = """
    You are a top-notch recruiter and data labeler.
    Your task is to thoroughly analyze the following part of a job posting and extract text spans including job-related skills or knowledge.
    A skill in a job context is typically indicated by a verb or an adjective-noun combination, reflecting an executable ability or a specific way of performing a task. 
    Knowledge, on the other hand, refers to non-executable information that an individual possesses, often indicated by specific fields, industries, or certifications, and is distinct from direct action or skills.
    
    Below you can see some examples. Pay attention to what is included in detected spans. We want not to include false positive skills and knowledge.

    Job posting:
    Ability to work in large collaborative teams to achieve organizational goals
    Detected spans:
    ["work in large collaborative teams"]

    Job posting:
    Work hands-on together with the other engineers within the Agile team 
    Detected spans:
    ["Work hands-on"]
    
    Job posting:
    Requirements At least 5 years of combined experience in Java or Kotlin and JavaScript or TypeScript programming and related test frameworks ( Selenium TestCafe etc.) .
    Detected spans:
    ["Java", "Kotlin", "JavaScript", "Typescript"]
    
    Job posting:
    A degree in Computer Science or related fields or equivalent practical experience . 
    Detected spans:
    ["degree in Computer Science"]
    
    Job posting:
    Experience in working on a cloud-based application running on Docker .
    Detected spans:
    ["Docker", "working on a cloud-based application"]
    
    Make sure to never include protected attributes like gender identity or veteran status in the returned list:
    
    Job posting:
    We do not discriminate on the basis of any protected attribute including race religion color national origin gender sexual orientation .
    Detected spans:
    []

    Output just the JSON array, wihtout any other text. 
    Start your response with "[" symbol, and finish with "]".
    Make sure your response doesn't include data from examples and anything else except from JSON array.
    Return empty array If there are no obvious skill and knowledge spans in the given posting.
    
    Job posting:
    {job_posting}
    Detected spans:
"""

template_collection = {
    'verbose': verbose_template,
    'short': short_template,
    'few_shot': few_shot_template
}

In [6]:
from langchain import LLMChain, PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

prompt_template = PromptTemplate(input_variables=["job_posting"], template=template_collection['few_shot'])

In [7]:
from langchain.llms.fireworks import Fireworks 

llm = Fireworks(model="accounts/fireworks/models/llama-v2-70b-chat", max_tokens=-1, top_p=0.6)

In [8]:
import ast

test_posting = test_tech_df.at[0, 'text']

def get_skills_for_posting(job_posting):
    prompt_val = prompt_template.invoke({"job_posting": test_posting})
    prompt = prompt_val.to_string()
    output = llm(prompt)
    array_start = output.find('[')
    array_end = output.find(']', array_start)
    array_string = output[array_start:array_end+1].strip()
    return ast.literal_eval(array_string)
     
llama_results = get_skills_for_posting(test_posting)
llama_results

InvalidRequestError: {'error': 'Credit limit reached. Please set up (additional) payment at https://app.fireworks.ai/users?tab=billing.'}

In [ ]:
def calculate_f1_score(real_values, predicted_values):
    tp = sum([1 for val in predicted_values if val in real_values])
    fp = sum([1 for val in predicted_values if val not in real_values])
    fn = sum([1 for val in real_values if val not in predicted_values])

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print("Precision:" + str(precision))
    print("Recall" + str(recall))

    return f1_score

In [ ]:
merged_skills_knowledge = test_tech_df.at[0, 'skills'] + test_tech_df.at[0, 'knowledge']

merged_skills_knowledge

In [ ]:
calculate_f1_score(merged_skills_knowledge, llama_results)

In [ ]:
from nltk.tokenize import sent_tokenize

def divide_into_chunks(text, min_sentences=3):
    sentences = sent_tokenize(text)
    
    chunks = []
    current_chunk = []

    for sentence in sentences:
        current_chunk.append(sentence)
        if len(current_chunk) == min_sentences:
            chunks.append(' '.join(current_chunk))
            current_chunk = []

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

chunks = divide_into_chunks(test_posting)

In [ ]:
def get_skills_for_posting_chunked(job_posting):
    chunks = divide_into_chunks(job_posting)
    responses = []
    
    for chunk in chunks:
        try:
            prompt_val = prompt_template.invoke({"job_posting": chunk})
            prompt = prompt_val.to_string()
            output = llm(prompt)
            array_start = output.find('[')
            array_end = output.find(']', array_start)

            array_string = output[array_start:array_end+1].strip()
            responses = responses + ast.literal_eval(array_string)
        except:
            print("ERROR processing chunk")
    
    return responses
    
responses = get_skills_for_posting_chunked(test_posting)

responses

In [ ]:
calculate_f1_score(merged_skills_knowledge, responses)

In [ ]:
gpt_4_result = [
"Full Stack Software Engineer",
"Java",
"JavaScript",
"solving business problems through innovation and engineering practices",
"software development lifecycle",
"experimentation and learning",
"collaborative",
"analytics products development",
"API development",
"financial instruments",
"BS/BA degree",
"Advanced knowledge of application data and infrastructure architecture disciplines",
"architecture and design across all systems",
"developmental toolsets",
"industry-wide technology strategies and best practices",
"modern programming languages",
"business analysis",
"development",
"maintenance",
"software improvement",
"HTML5",
"JavaScript",
"developing multithreaded scalable concurrent applications",
"Node JS",
"React"
]

f1 = calculate_f1_score(merged_skills_knowledge, gpt_4_result)
print("F1 Score:", f1)

In [ ]:
f1_scores = []

for index, row in test_tech_df.iterrows():
    try:
        row_merged_skills_knowledge = test_tech_df.at[index, 'skills'] + test_tech_df.at[index, 'knowledge']
        res = get_skills_for_posting_chunked(row['text'])
        f1_score = calculate_f1_score(row_merged_skills_knowledge, res)
        print(f1_score)
        f1_scores.append(f1_score)
    except Exception as e:
        print(f"Failed to parse skills for row {index}: {e}")
        
f1_scores

In [ ]:
sum(f1_scores) / len(f1_scores)

In [ ]:
# Check baseline performance:

In [ ]:
from transformers import pipeline

token_skill_classifier = pipeline(model="jjzha/jobbert_skill_extraction", aggregation_strategy="first")
token_knowledge_classifier = pipeline(model="jjzha/jobbert_knowledge_extraction", aggregation_strategy="first")

In [ ]:
def aggregate_span(results):
    new_results = []
    current_result = results[0]

    for result in results[1:]:
        if result["start"] == current_result["end"] + 1:
            current_result["word"] += " " + result["word"]
            current_result["end"] = result["end"]
        else:
            new_results.append(current_result)
            current_result = result

    new_results.append(current_result)

    return new_results

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def ner(text):
    output_skills = []
    output_knowledge = []

    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]

    for sentence in sentences:
        skills, knowledge = process_sentence(sentence)
        output_skills.extend(skills)
        output_knowledge.extend(knowledge)

    return {"text": text, "entities": output_skills}, {"text": text, "entities": output_knowledge}

def process_sentence(sentence):
    skills = token_skill_classifier(sentence)
    for result in skills:
        if result.get("entity_group"):
            result["entity"] = "Skill"
            del result["entity_group"]

    knowledge = token_knowledge_classifier(sentence)
    for result in knowledge:
        if result.get("entity_group"):
            result["entity"] = "Knowledge"
            del result["entity_group"]

    if len(skills) > 0:
        skills = aggregate_span(skills)
    if len(knowledge) > 0:
        knowledge = aggregate_span(knowledge)

    return skills, knowledge

In [ ]:
f1_scores_baseline = []

for index, row in test_tech_df.iterrows():
    try:
        row_merged_skills_knowledge = test_tech_df.at[index, 'skills'] + test_tech_df.at[index, 'knowledge']
        ner_results = ner(row['text'])
        detected_entities = [entity for entity_group in ner_results for entity in entity_group["entities"]]
        detected_words = [entity['word'] for entity in detected_entities]
        f1_score = calculate_f1_score(row_merged_skills_knowledge, detected_words)
        f1_scores_baseline.append(f1_score)
    except Exception as e:
        print(f"Failed to parse skills for row {index}: {e}")
        
f1_scores_baseline

In [ ]:
sum(f1_scores_baseline) / len(f1_scores_baseline)